In [1]:
import numpy as np

# Part 1

In [2]:
test = ['Tile 2311:','..##.#..#.','##..#.....','#...##..#.',
        '####.#...#','##.##.###.','##...#.###','.#.#.#..##',
        '..#....#..','###...#.#.','..###..###',
        'Tile 1951:','#.##...##.','#.####...#','.....#..##',
        '#...######','.##.#....#','.###.#####','###.##.##.',
        '.###....#.','..#.#..#.#','#...##.#..',
        'Tile 1171:','####...##.','#..##.#..#','##.#..#.#.',
        '.###.####.','..###.####','.##....##.','.#...####.',
        '#.##.####.','####..#...','.....##...',
        'Tile 1427:','###.##.#..','.#..#.##..','.#.##.#..#',
        '#.#.#.##.#','....#...##','...##..##.','...#.#####',
        '.#.####.#.','..#..###.#','..##.#..#.',
        'Tile 1489:','##.#.#....','..##...#..','.##..##...',
        '..#...#...','#####...#.','#..#.#.#.#','...#.#.#..',
        '##.#...##.','..##.##.##','###.##.#..',
        'Tile 2473:','#....####.','#..#.##...','#.##..#...',
        '######.#.#','.#...#.#.#','.#########','.###.#..#.',
        '########.#','##...##.#.','..###.#.#.',
        'Tile 2971:','..#.#....#','#...###...','#.#.###...',
        '##.##..#..','.#####..##','.#..####.#','#..#.#..#.',
        '..####.###','..#.#.###.','...#.#.#.#',
        'Tile 2729:','...#.#.#.#','####.#....','..#.#.....',
        '....#..#.#','.##..##.#.','.#.####...','####.#.#..',
        '##.####...','##..#.##..','#.##...##.',
        'Tile 3079:','#.#.#####.','.#..######','..#.......',
        '######....','####.#..#.','.#...#.##.','#.#####.##',
        '..#.###...','..#.......','..#.###...']

In [3]:
data = np.genfromtxt('day20_input.txt', delimiter='\n', comments=None, dtype=str)

In [4]:
def get_tiles_and_shape(data):
    tiles = {}
    edges = {}
    tile = []
    edge  = []
    idx = None
    for line in data:
        if line[0] == 'T':
            if idx is not None:
                tiles[idx] = tile
                
                edge.append(tile[0])        #0 top
                edge.append(tile[0][::-1])  #1 top flip
                edge.append(tile[-1])       #2 bottom
                edge.append(tile[-1][::-1]) #3 bottom flip

                string0 = ''
                string1 = ''
                for i in range(0, len(tile)):
                    string0 += tile[i][0]
                    string1 += tile[i][-1]

                edge.append(string0)       #4 left
                edge.append(string0[::-1]) #5 left flip
                edge.append(string1)       #6 right
                edge.append(string1[::-1]) #7 right flip

                edges[idx] = edge
            
            parts = line.split(' ')
            idx = int(parts[1][:-1])
            
            tile = []
            edge = []
            
        else:
            tile.append(line)
            
    tiles[idx] = tile
    
    edge.append(tile[0])
    edge.append(tile[0][::-1])
    edge.append(tile[-1])
    edge.append(tile[-1][::-1])

    string0 = ''
    string1 = ''
    for i in range(0, len(tile)):
        string0 += tile[i][0]
        string1 += tile[i][-1]

    edge.append(string0)
    edge.append(string0[::-1])
    edge.append(string1)
    edge.append(string1[::-1])
    
    edges[idx] = edge
    
    shape = np.sqrt(len(tiles.keys()))
    
    return tiles, edges, shape

def get_neighbours(edges):
    keys = list(edges.keys())
    
    neighbours = {}
    for i in range(0, len(keys)):
        neighbour = []
        key_i = keys[i]
        for j in range(0, len(keys)):
            if i == j:
                continue
            key_j = keys[j]
            for k in range(0, len(edges[key_i])):
                if edges[key_i][k] in edges[key_j]:
                    neighbour.append([key_j,k])
                    break
                    
        neighbours[key_i] = np.array(neighbour)
    
    return neighbours

def get_corners(neighbours):   
    corners = []
    for key in neighbours.keys():
        if len(neighbours[key]) == 2:
            corners.append(key)
            
    return corners

def part1_wrapper(data):
    tiles, edges, shape = get_tiles_and_shape(data)
    neighbours = get_neighbours(edges)
    corners = get_corners(neighbours)
    return np.prod(corners, dtype=np.int64)

print(part1_wrapper(test))

20899048083289


In [5]:
print('Part 1 Result:', part1_wrapper(data))

Part 1 Result: 8425574315321


# Part 2

In [6]:
def picture_order(shape, corners, neighbours):   
    shape = int(shape)
    image_idx = np.zeros((shape,shape), dtype=int)
    
    for i in range(0, shape):
        for j in range(0, shape):
            if i == 0 and j == 0:
                image_idx[i,j] = corners[0]
                
            elif i == 0:
                cut = 4
                if j == shape-1:
                    cut -= 1
                
                all_idx = neighbours[image_idx[i,j-1]][:,0]
                poss_idx = []
                for idx in all_idx:
                    if idx in image_idx:
                        continue
                    elif len(neighbours[idx]) >= cut:
                        continue
                    else:
                        poss_idx.append(idx)
                image_idx[i,j] = poss_idx[0]
                
            elif j == 0:
                cut = 4
                
                all_idx = neighbours[image_idx[i-1,j]][:,0]
                poss_idx = []
                for idx in all_idx:
                    if idx in image_idx:
                        continue
                    elif len(neighbours[idx]) >= cut:
                        continue
                    else:
                        poss_idx.append(idx)
                image_idx[i,j] = poss_idx[0]
                
            else:
                i1_j = neighbours[image_idx[i-1,j]][:,0]
                for k in range(len(i1_j)-1, -1, -1):
                    if i1_j[k] in image_idx:
                        i1_j = np.delete(i1_j, k)
                i_j1 = neighbours[image_idx[i,j-1]][:,0]
                for k in range(len(i_j1)-1, -1, -1):
                    if i_j1[k] in image_idx:
                        i_j1 = np.delete(i_j1, k)
                next_idx = np.intersect1d(i1_j, i_j1)
                image_idx[i,j] = next_idx
    
    return image_idx

def get_bordered_image(tiles, shape, neighbours, image_idx):
    #0 top
    #1 top flip
    #2 bottom
    #3 bottom flip
    #4 left
    #5 left flip
    #6 right
    #7 right flip
    
    shape = int(shape)
    bordered_image = np.empty((shape*10,shape*10), dtype=str)
    
    for i in range(0, shape):
        for j in range(0, shape):           
            tile_idx = image_idx[i,j]
            neighbour = neighbours[tile_idx]
            
            if i < shape-1 and j < shape-1:
                tile_r = image_idx[i,j+1]
                tile_d = image_idx[i+1,j]
                
                idx_r = np.where(neighbour[:,0]==tile_r)[0]
                idx_d = np.where(neighbour[:,0]==tile_d)[0]
                orient_r = neighbour[idx_r,1]
                orient_d = neighbour[idx_d,1]
                
            elif i == shape-1 and j < shape-1:
                tile_r = image_idx[i,j+1]
                tile_u = image_idx[i-1,j]
                
                idx_r = np.where(neighbour[:,0]==tile_r)[0]
                idx_u = np.where(neighbour[:,0]==tile_u)[0]
                orient_r = neighbour[idx_r,1]
                orient_u = neighbour[idx_u,1]
                
                if orient_u == 0 or orient_u == 1 or orient_u == 4 or orient_u  == 5:
                    orient_d = orient_u + 2
                else:
                    orient_d = orient_u - 2
                    
            elif i < shape-1 and j == shape-1:
                tile_l = image_idx[i,j-1]
                tile_d = image_idx[i+1,j]
                
                idx_l = np.where(neighbour[:,0]==tile_l)[0]
                idx_d = np.where(neighbour[:,0]==tile_d)[0]
                orient_l = neighbour[idx_l,1]
                orient_d = neighbour[idx_d,1]
                
                if orient_l == 0 or orient_l == 1 or orient_l == 4 or orient_l  == 5:
                    orient_r = orient_l + 2
                else:
                    orient_r = orient_l - 2
                    
            else: #i == shape-1 and j == shape-1
                tile_l = image_idx[i,j-1]
                tile_u = image_idx[i-1,j]
                
                idx_l = np.where(neighbour[:,0]==tile_l)[0]
                idx_u = np.where(neighbour[:,0]==tile_u)[0]
                orient_l = neighbour[idx_l,1]
                orient_u = neighbour[idx_u,1]
                
                if orient_l == 0 or orient_l == 1 or orient_l == 4 or orient_l  == 5:
                    orient_r = orient_l + 2
                else:
                    orient_r = orient_l - 2
                    
                if orient_u == 0 or orient_u == 1 or orient_u == 4 or orient_u  == 5:
                    orient_d = orient_u + 2
                else:
                    orient_d = orient_u - 2
                    
            tile = tiles[tile_idx]
            
            new_tile = []
            for string in tile:
                chars = []
                for c in string:
                    chars.append(c)
                new_tile.append(chars)
            tile = np.array(new_tile)
            
            if orient_r == 0 or orient_r == 1: #rotate clockwise
                tile = np.rot90(tile, 3)
                if orient_d == 4 or orient_d == 5: #flip t-b
                    tile = np.flip(tile, 0)
                elif orient_d == 6 or orient_d == 7: #do nothing
                    pass
                
            elif orient_r == 2 or orient_r == 3: #rotate anticlockwise
                tile = np.rot90(tile, 1)
                if orient_d == 4 or orient_d == 5: #do nothing
                    pass
                elif orient_d == 6 or orient_d == 7: #flip t-b
                    tile = np.flip(tile, 0)
                    
            elif orient_r == 4 or orient_r == 5: #flip l-r
                tile = np.flip(tile, 1)
                if orient_d == 0 or orient_d == 1: #flip t-b
                    tile = np.flip(tile, 0)
                elif orient_d == 2 or orient_d == 3: #do nothing
                    pass
                
            elif orient_r == 6 or orient_r == 7: #do nothing
                if orient_d == 0 or orient_d == 1: #flip t-b
                    tile = np.flip(tile, 0)
                elif orient_d == 2 or orient_d == 3: #do nothing
                    pass
                
            bordered_image[i*10:(i+1)*10, j*10:(j+1)*10] = tile
            
    return bordered_image

def get_image(bordered_image, shape):
    shape = int(shape)
    remove = []
    for i in range(0, shape):
        remove.append(i*10)
        remove.append(((i+1)*10)-1)
        
    image = np.delete(bordered_image, remove, 0)
    image = np.delete(image, remove, 1)
    
    return image

def get_roughness(image, monster, monster_mask):
    for i in range(0, len(image)-len(monster)):
        for j in range(0, len(image[i])-len(monster[0])):
            count = 0
            for xy in monster_mask:
                if image[i+xy[0],j+xy[1]] == '#':
                    count +=  1
                else:
                    break
            if count == len(monster_mask):
                for xy in monster_mask:
                    image[i+xy[0],j+xy[1]] = 'O'

    #for i in range(0, len(image)):
    #    string = ''
    #    for j in range(0, len(image[i])):
    #        string += image[i][j]
    #    print(string)
    #print()

    rough = np.where(image == '#')[0]
    return  len(rough)
    
def mask_monsters(image):
    monster = ['                  # ',
               '#    ##    ##    ###',
               ' #  #  #  #  #  #   ']
    
    new_monster = []
    for string in monster:
        chars = []
        for c in string:
            chars.append(c)
        new_monster.append(chars)
    monster = np.array(new_monster)
    mask = np.where(monster == '#')
    zipped= zip(mask[0], mask[1])
    monster_mask = []
    for z in zipped:
        monster_mask.append(list(z))
    monster_mask  = np.array(monster_mask)
    
    rough = np.where(image == '#')[0]
    original_rough = len(rough)
    
    for h in range(0, 4):
        new_roughness = get_roughness(image, monster, monster_mask)
        if new_roughness != original_rough:
            return new_roughness
        
        image = np.flip(image, 0)
        new_roughness = get_roughness(image, monster, monster_mask)
        if new_roughness != original_rough:
            return new_roughness
        
        image = np.flip(image, 0)
        image = np.flip(image, 1)
        new_roughness = get_roughness(image, monster, monster_mask)
        if new_roughness != original_rough:
            return new_roughness
        
        image = np.flip(image, 1)
        image = np.rot90(image)
            
    return -1

def part2_wrapper(data):
    tiles, edges, shape = get_tiles_and_shape(data)
    neighbours = get_neighbours(edges)
    corners = get_corners(neighbours)
    image_idx = picture_order(shape, corners, neighbours)
    bordered_image = get_bordered_image(tiles, shape, neighbours, image_idx)   
    image = get_image(bordered_image, shape)
    roughness = mask_monsters(image)
    return roughness
    
print(part2_wrapper(test))

273


In [7]:
print('Part 2 Result:', part2_wrapper(data))

Part 2 Result: 1841
